In [89]:
import pandas as pd
import matplotlib.pyplot as plt
import mplfinance as mpf
from datetime import timedelta

class Trade:
    def __init__(self, entry_time, entry_price, trade_type, cash, size=1, commission_rate=0):
        self.entry_time = entry_time
        self.entry_price = entry_price
        self.trade_type = trade_type
        self.size = size
        self.exit_time = None
        self.exit_price = None
        self.exit_type = None
        self.in_pos = None
        self.profit = 0
        self.pr_profit = 0
        self.commission_rate = commission_rate
        self.cash = cash

    def close(self, exit_time, exit_price, exit_type):
        self.exit_time = exit_time
        self.exit_price = exit_price
        self.exit_type = exit_type
        if self.trade_type == 'long':
            self.pr_profit = ((self.exit_price - self.entry_price) / self.entry_price - self.commission_rate) * self.size
        elif self.trade_type == 'short':
            self.pr_profit = ((self.entry_price - self.exit_price) / self.entry_price - self.commission_rate) * self.size
        self.profit = self.pr_profit * self.size * self.cash
        self.in_pos = (self.exit_time - self.entry_time).total_seconds() / 86400
        return self.profit

class LimitOrder:
    def __init__(self, entry_time, entry_price, expiry_time, order_type):
        self.entry_time = entry_time
        self.entry_price = entry_price
        self.expiry_time = expiry_time
        self.order_type = order_type
        self.filled = False
        self.cancelled = False

class Backtest:
    def __init__(self, data, sym, initial_cash=10000, stop_loss=0.035, commission_rate=0.001):
        self.data = data
        self.sym = sym
        self.cash = initial_cash
        self.trades = []
        self.current_trade = None
        self.current_limit_order = None
        self.log = []
        self.commission_rate = commission_rate
        self.stop_loss = stop_loss

    def log_trade(self, action, price, time, details=""):
        self.log.append({'time': time, 'action': action, 'price': price, 'cash': self.cash, 'details': details})
        print(f"{time} - {action} at {price} - Cash: {self.cash} - {details}")

    def run(self):
        for i in range(len(self.data)):
            row = self.data.iloc[i]
            self.process_row(row, i)

    def process_row(self, row, idx):
        if self.current_limit_order:
            self.check_limit_order(row)
        if not self.current_limit_order:
            if self.current_trade:
                self.check_exit_conditions(row, idx)
            if not self.current_trade:
                self.check_entry_conditions(row, idx)

    def check_entry_conditions(self, row, idx):
        if row['signal type'] == 'long':
            self.current_trade = Trade(row.name, row['Close'], 'long', cash=self.cash, commission_rate=self.commission_rate)
            self.trades.append(self.current_trade)
            self.log_trade('BUY', row['Close'], row.name, "Entered long position")

        elif row['signal type'] == 'short':
            self.current_trade = Trade(row.name, row['Close'], 'short', cash=self.cash, commission_rate=self.commission_rate)
            self.trades.append(self.current_trade)
            self.log_trade('SELL', row['Close'], row.name, "Entered long position")

    def check_limit_order(self, row):
        if row.name > self.current_limit_order.expiry_time:
            self.log_trade('LIMIT ORDER EXPIRED', self.current_limit_order.entry_price, row.name, "Limit order expired")
            self.current_limit_order = None
        elif row['Low'] <= self.current_limit_order.entry_price <= row['High']:
            trade_type = self.current_limit_order.order_type
            self.current_trade = Trade(self.current_limit_order.entry_time, self.current_limit_order.entry_price, trade_type, cash=self.cash, commission_rate=self.commission_rate)
            self.trades.append(self.current_trade)
            self.log_trade('TRADE FILLED', self.current_limit_order.entry_price, row.name, f"Entered {trade_type} position from limit order")
            self.current_limit_order = None

    def check_exit_conditions(self, row, idx):
        if self.current_trade.trade_type == 'long':
            if row['color'] == 'blue' or row['color'] == 'green':
                profit_percentage = (row['Close'] - self.current_trade.entry_price) / self.current_trade.entry_price
                if profit_percentage > 0.01:
                    profit = self.current_trade.close(row.name, row['Close'], 'blue-green')
                    self.cash += profit
                    self.log_trade('SELL', row['Close'], row.name, f"Exited long position with >1% profit due to blue signal on {row.name}")
                    self.current_trade = None
                    return

            stop_loss_price = self.current_trade.entry_price * (1 - self.stop_loss)
            if row['Low'] <= stop_loss_price:
                profit = self.current_trade.close(row.name, stop_loss_price, 'stop loss')
                self.cash += profit
                self.log_trade('STOP LOSS HIT', stop_loss_price, row.name, f"Exited long position due to stop loss at {stop_loss_price} on {row.name}")
                self.current_trade = None
                return

        elif self.current_trade.trade_type == 'short':
            if row['Low'] <= self.current_trade.entry_price * 0.99:
                profit = self.current_trade.close(row.name, self.current_trade.entry_price * 0.99, '1pr exit')
                self.cash += profit
                self.log_trade('BUY', row['Close'], row.name, f"Exited short position with 1% profit on {row.name}")
                self.current_trade = None
                return

            stop_loss_price = self.current_trade.entry_price * (1 + self.stop_loss)
            if row['High'] >= stop_loss_price:
                profit = self.current_trade.close(row.name, stop_loss_price, 'stop loss')
                self.cash += profit
                self.log_trade('STOP LOSS HIT', stop_loss_price, row.name, f"Exited short position due to stop loss at {stop_loss_price} on {row.name}")
                self.current_trade = None
                return

    def report(self):
        trades_data = [
            {
                'Entry Time': trade.entry_time,
                'Trade Type': trade.trade_type,
                'Entry Price': round(trade.entry_price, 5),
                'Exit Time': trade.exit_time,
                'Exit Price': round(trade.exit_price, 5),
                'Profit': round(trade.profit, 5),
                'pr profit': round(trade.pr_profit * 100, 3),
                'Exit type': trade.exit_type,
                'In position time': trade.exit_time - trade.entry_time
            }
            for trade in self.trades
        ]
        trades_df = pd.DataFrame(trades_data)
        trades_df.to_excel(f'backtest report/trades_report-{sym}.xlsx', index=False)

        total_profit = sum(trade.profit for trade in self.trades)
        total_in_pos = round(sum(trade.in_pos for trade in self.trades), 3)
        print(f'Total Profit: {total_profit}')
        print(f'Final Cash: {self.cash}')
        print(f'In position: {total_in_pos}')

        # for trade in self.trades:
            # print(f'{trade.entry_time} - {trade.trade_type.upper()} - Entry: {trade.entry_price} Exit: {trade.exit_price} Exit Time: {trade.exit_time} Profit: {trade.profit}  Exit type: {trade.exit_type}')

        return {'symbol': self.sym, 'total_profit': total_profit, 'final_cash': self.cash, 'In position time': total_in_pos}

In [90]:
overall_results = []
Symbols = ['ADA','ATOM','BTC','CAKE','DOGE','ETH','FET','FTM','LTC','MANA','SAND','SHIB','SOL','XLM', 'XRP']
# Symbols = ['ADA', 'ATOM']

for sym in Symbols:
    try:
        df = pd.read_csv(f'backtest\Backtest-{sym}.csv')
        df['Date'] = pd.to_datetime(df['Date'])
        df.set_index('Date', inplace=True)

        backtest = Backtest(df, sym)
        backtest.run()
        result = backtest.report()
        overall_results.append(result)
    except:
        print(sym)

results_df = pd.DataFrame(overall_results)
results_df.to_excel('backtest report/overall_results.xlsx', index=False)

C:\Users\mhjo.DESKTOP-OQN6B1J\AppData\Local\Temp\ipykernel_19984\337898067.py:7: DtypeWarning: Columns (7,8) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(f'backtest\Backtest-{sym}.csv')


2020-03-28 09:00:00 - BUY at 0.0288 - Cash: 10000 - Entered long position
2020-03-28 21:00:00 - SELL at 0.02942 - Cash: 10205.27777777778 - Exited long position with >1% profit due to blue signal on 2020-03-28 21:00:00
2020-04-25 03:00:00 - SELL at 0.04211 - Cash: 10205.27777777778 - Entered long position
2020-04-26 12:00:00 - STOP LOSS HIT at 0.04358385 - Cash: 9837.88777777778 - Exited short position due to stop loss at 0.04358385 on 2020-04-26 12:00:00
2020-06-26 12:00:00 - BUY at 0.08131 - Cash: 9837.88777777778 - Entered long position
2020-06-27 16:05:00 - STOP LOSS HIT at 0.07846415 - Cash: 9483.72381777778 - Exited long position due to stop loss at 0.07846415 on 2020-06-27 16:05:00
2020-06-28 03:00:00 - BUY at 0.07663 - Cash: 9483.72381777778 - Entered long position
2020-06-28 12:00:00 - SELL at 0.0789 - Cash: 9755.175146372314 - Exited long position with >1% profit due to blue signal on 2020-06-28 12:00:00
2020-08-27 21:00:00 - BUY at 0.10595 - Cash: 9755.175146372314 - Entered

C:\Users\mhjo.DESKTOP-OQN6B1J\AppData\Local\Temp\ipykernel_19984\337898067.py:7: DtypeWarning: Columns (7,8) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(f'backtest\Backtest-{sym}.csv')


2020-02-17 09:00:00 - BUY at 4.199 - Cash: 10000 - Entered long position
2020-02-17 11:05:00 - STOP LOSS HIT at 4.052035 - Cash: 9640.0 - Exited long position due to stop loss at 4.052035 on 2020-02-17 11:05:00
2020-03-06 12:00:00 - SELL at 3.823 - Cash: 9640.0 - Entered long position
2020-03-06 13:50:00 - BUY at 3.751 - Cash: 9726.76 - Exited short position with 1% profit on 2020-03-06 13:50:00
2020-03-09 09:00:00 - SELL at 3.11 - Cash: 9726.76 - Entered long position
2020-03-09 09:55:00 - BUY at 3.081 - Cash: 9814.30084 - Exited short position with 1% profit on 2020-03-09 09:55:00
2020-03-12 12:00:00 - SELL at 1.912 - Cash: 9814.30084 - Entered long position
2020-03-12 12:05:00 - BUY at 1.9 - Cash: 9902.62954756 - Exited short position with 1% profit on 2020-03-12 12:05:00
2020-05-04 09:00:00 - BUY at 2.667 - Cash: 9902.62954756 - Entered long position
2020-05-05 00:00:00 - SELL at 2.768 - Cash: 10267.742135224124 - Exited long position with >1% profit due to blue signal on 2020-05-0

C:\Users\mhjo.DESKTOP-OQN6B1J\AppData\Local\Temp\ipykernel_19984\337898067.py:7: DtypeWarning: Columns (7,8) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(f'backtest\Backtest-{sym}.csv')


...

2020-10-14 21:00:00 - BUY at 2.2786 - Cash: 10000 - Entered long position
2020-10-15 09:25:00 - STOP LOSS HIT at 2.198849 - Cash: 9640.0 - Exited long position due to stop loss at 2.198849 on 2020-10-15 09:25:00
2020-10-20 12:00:00 - BUY at 1.8613 - Cash: 9640.0 - Entered long position
2020-10-21 09:00:00 - SELL at 1.9235 - Cash: 9952.504737549025 - Exited long position with >1% profit due to blue signal on 2020-10-21 09:00:00
2020-10-26 03:00:00 - BUY at 1.8196 - Cash: 9952.504737549025 - Entered long position
2020-10-26 16:00:00 - STOP LOSS HIT at 1.755914 - Cash: 9594.21456699726 - Exited long position due to stop loss at 1.755914 on 2020-10-26 16:00:00
2020-10-29 00:00:00 - SELL at 1.5212 - Cash: 9594.21456699726 - Entered long position
2020-10-29 04:00:00 - STOP LOSS HIT at 1.574442 - Cash: 9248.82284258536 - Exited short position due to stop loss at 1.574442 on 2020-10-29 04:00:00
2020-11-02 15:00:00 - BUY at 1.4923 - Cash: 9248.82284258536 - Entered long position
2020-11-03 03:1

C:\Users\mhjo.DESKTOP-OQN6B1J\AppData\Local\Temp\ipykernel_19984\337898067.py:7: DtypeWarning: Columns (7,8) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(f'backtest\Backtest-{sym}.csv')


2020-03-28 12:00:00 - SELL at 0.03919 - Cash: 10000 - Entered long position
2020-03-28 17:55:00 - BUY at 0.03877 - Cash: 10090.0 - Exited short position with 1% profit on 2020-03-28 17:55:00
2020-04-16 00:00:00 - SELL at 0.0454 - Cash: 10090.0 - Entered long position
2020-04-16 00:25:00 - BUY at 0.04475 - Cash: 10180.810000000001 - Exited short position with 1% profit on 2020-04-16 00:25:00
2020-05-10 15:00:00 - SELL at 0.0644 - Cash: 10180.810000000001 - Entered long position
2020-05-10 15:45:00 - BUY at 0.06387 - Cash: 10272.437290000002 - Exited short position with 1% profit on 2020-05-10 15:45:00
2020-06-28 03:00:00 - BUY at 0.06181 - Cash: 10272.437290000002 - Entered long position
2020-06-28 18:00:00 - SELL at 0.06431 - Cash: 10677.649292525568 - Exited long position with >1% profit due to blue signal on 2020-06-28 18:00:00
2022-07-21 12:00:00 - BUY at 0.1106 - Cash: 10677.649292525568 - Entered long position
2022-07-21 21:00:00 - SELL at 0.1136 - Cash: 10956.60046671927 - Exited

C:\Users\mhjo.DESKTOP-OQN6B1J\AppData\Local\Temp\ipykernel_19984\337898067.py:7: DtypeWarning: Columns (7,8) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(f'backtest\Backtest-{sym}.csv')


2023-08-05 00:00:00 - BUY at 0.6344 - Cash: 10000 - Entered long position
2023-08-05 01:35:00 - STOP LOSS HIT at 0.612196 - Cash: 9640.0 - Exited long position due to stop loss at 0.612196 on 2023-08-05 01:35:00
2023-08-17 06:00:00 - SELL at 0.5908 - Cash: 9640.0 - Entered long position
2023-08-17 09:40:00 - BUY at 0.5847 - Cash: 9726.76 - Exited short position with 1% profit on 2023-08-17 09:40:00
Total Profit: -273.23999999999967
Final Cash: 9726.76
In position: 0.219


In [91]:
df = pd.read_csv(f'backtest\Backtest-ETH.csv')
df['Date'] = pd.to_datetime(df['Date'])
df.set_index('Date', inplace=True)

UnicodeDecodeError: 'utf-8' codec can't decode byte 0xb7 in position 249861: invalid start byte

In [81]:
i = df.index
(i[1] - i[0]).total_seconds()

300.0